In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path

import util
import lines
import group
import label
import records

In [2]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
file = test_files[-2]
display(test_files, file)

['.././test_files/\\Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/\\LS_index_1920.pdf',
 '.././test_files/\\LS_index_1920_split.pdf',
 '.././test_files/\\LS_index_1921.pdf',
 '.././test_files/\\LS_index_1937.pdf',
 '.././test_files/\\LS_index_1944.pdf',
 '.././test_files/\\LS_index_1944_split.pdf',
 '.././test_files/\\LS_index_1976.pdf',
 '.././test_files/\\LS_index_1986.pdf',
 '.././test_files/\\LS_index_1986_split.pdf']

'.././test_files/\\LS_index_1986.pdf'

In [3]:
p_start = 3
pages, dicts = util.read_pdf(file, p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/\LS_index_1986.pdf
...
Finished reading 87 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,(109) 3 1987—List No. 3,"[(109) , 3 , 1987—List No. 3 ]",36.48,38.91,366.33,52.99,3
1,ANTIGUA AND BARBUDA,[ANTIGUA AND BARBUDA ],37.44,64.28,143.45,76.75,3
2,5/VÍ/1986 Social Security (Benefits) (Invalid...,"[5/VÍ/1986, Social Security (Benefits) (Inval...",37.44,76.20,367.58,88.21,3
3,1986. S.I. 1986 No. 25.,"[1986., S.I., 1986, No. 25. ]",50.16,84.84,133.63,96.85,3
4,"[Amendments to Regulations 2 (definitions), 5,...","[[Amendments to Regulations 2 (definitions), 5...",49.68,97.08,367.75,109.09,3
...,...,...,...,...,...,...,...
4551,(195) 89 1987—List No. 3,"[(195) , 89 , 1987—List No. 3 ]",43.68,45.00,373.53,57.01,89
4552,Spain—Uruguay,[Spain—Uruguay ],43.92,70.04,103.60,82.51,89
4553,19/11/1986 Additional international technical ...,[19/11/1986 Additional international technical...,44.64,81.96,373.93,93.97,89
4554,matters between the Republic of Uruguay and th...,[matters between the Republic of Uruguay and t...,55.68,90.36,379.08,102.37,89


In [3]:
pdf_df = util.ocr(file, start_page=3)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
lines_df

Converting pdf pages .././test_files/\LS_index_1986.pdf to images.
Starting OCR...
Done with page 3
Done with page 4
Done with page 5
Done with page 6
Done with page 7
Done with page 8
Done with page 9
Done with page 10
Done with page 11
Done with page 12
Done with page 13
Done with page 14
Done with page 15
Done with page 16
Done with page 17
Done with page 18
Done with page 19
Done with page 20
Done with page 21
Done with page 22
Done with page 23
Done with page 24
Done with page 25
Done with page 26
Done with page 27
Done with page 28
Done with page 29
Done with page 30
Done with page 31
Done with page 32
Done with page 33
Done with page 34
Done with page 35
Done with page 36
Done with page 37
Done with page 38
Done with page 39
Done with page 40
Done with page 41
Done with page 42
Done with page 43
Done with page 44
Done with page 45
Done with page 46
Done with page 47
Done with page 48
Done with page 49
Done with page 50
Done with page 51
Done with page 52
Done with page 53
Done w

,line_text,x0,y0,x1,y1,page,dx
0,(109) 3 1987—List No. 3,203,233,2019,284,3,1816
1,ANTIGUA AND BARBUDA,208,373,784,408,3,576
2,5/V1/1986 Social Security (Benefits) (Invalidi...,208,441,2029,484,3,1821
3,1986. S.I. 1986 No. 25.,279,491,711,524,3,432
4,"{Amendments to Regulations 2 (definitions), 5,...",276,557,2027,601,3,1751
...,...,...,...,...,...,...,...
4582,(195) 89 1987—List No. 3,243,264,2059,317,89,1816
4583,Spain—Uruguay,244,405,561,448,89,317
4584,19/11/1986 Additional international technical ...,248,474,2066,517,89,1818
4585,matters between the Republic of Uruguay and th...,311,521,2066,565,89,1755


In [4]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

2

,x0,lines,last_x0,count,page
1,"[203, 208, 208, 201, 208, 209, 208, 215, 209, ...","[0, 1, 2, 5, 9, 13, 14, 17, 24, 43, 46]",213,11,3
0,"[279, 276, 275, 277, 275, 275, 277, 273, 275, ...","[3, 4, 6, 7, 8, 10, 11, 12, 15, 16, 18, 19, 20...",273,44,3
1,"[431, 416, 416, 419, 419, 419, 421, 420, 421, ...","[55, 56, 57, 73, 75, 79, 82, 83, 87, 89, 93, 9...",425,15,4
0,"[481, 484, 485, 484, 483, 483, 484, 484, 484, ...","[58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 6...",495,37,4
1,"[220, 217, 217, 220, 216, 216, 217, 216, 215, ...","[107, 108, 109, 112, 117, 125, 128, 131, 132, ...",219,16,5
...,...,...,...,...,...
0,"[285, 287, 287, 287, 289, 287, 288, 285, 287, ...","[4484, 4485, 4488, 4489, 4490, 4491, 4492, 449...",293,41,87
1,"[408, 403, 405, 405, 401, 405, 400, 400, 399, ...","[4533, 4534, 4535, 4542, 4544, 4545, 4554, 455...",400,14,88
0,"[469, 471, 467, 467, 467, 472, 468, 471, 469, ...","[4536, 4537, 4538, 4539, 4540, 4541, 4543, 454...",468,35,88
1,"[243, 244, 248]","[4582, 4583, 4584]",248,3,89


,x1,lines,last_x1,count,page
14,"[2019, 2029, 2027, 2026, 2034, 2027, 2030, 203...","[0, 2, 4, 5, 7, 9, 11, 14, 15, 16, 17, 18, 19,...",2025,40,3
10,"[1295, 1284]","[8, 47]",1284,2,3
5,1750,23,1750,1,3
1,1527,45,1527,1,3
4,1252,25,1252,1,3
...,...,...,...,...,...
9,654,4553,654,1,88
0,605,4581,605,1,88
2,"[2059, 2066, 2066]","[4582, 4584, 4585]",2066,3,89
0,1366,4586,1366,1,89


In [5]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
l.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,(109) 3 1987—List No. 3,203,233,2019,284,3,1816,0,2,start
1,ANTIGUA AND BARBUDA,208,373,784,408,3,576,0,0,country
2,5/V1/1986 Social Security (Benefits) (Invalidi...,208,441,2029,484,3,1821,0,2,start
3,1986. S.I. 1986 No. 25.,279,491,711,524,3,432,1,0,end
4,"{Amendments to Regulations 2 (definitions), 5,...",276,557,2027,601,3,1751,1,2,middle
5,5/V1/1986 Social Security (Benefits) (Maternit...,201,628,2026,671,3,1825,0,2,start
6,26.,275,676,325,709,3,50,1,0,end
7,[Amend Regulations 8 and 10 of the principal R...,277,743,2034,788,3,1757,1,2,middle
8,time during which maternity allowance may be p...,275,791,1295,835,3,1020,1,0,end
9,5/VI/1986 Social Security (Benefits) (Age Pens...,208,860,2027,904,3,1819,0,2,start


In [7]:
cli_df = label.correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)

lli_df.head(50)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,(109) 3 1987—List No. 3,203,233,2019,284,3,1816,0,2,start,start
1,ANTIGUA AND BARBUDA,208,373,784,408,3,576,0,0,country,country
2,5/V1/1986 Social Security (Benefits) (Invalidi...,208,441,2029,484,3,1821,0,2,start,start
3,1986. S.I. 1986 No. 25.,279,491,711,524,3,432,1,0,end,end
4,"{Amendments to Regulations 2 (definitions), 5,...",276,557,2027,601,3,1751,1,2,middle,middle
5,5/V1/1986 Social Security (Benefits) (Maternit...,201,628,2026,671,3,1825,0,2,start,start
6,26.,275,676,325,709,3,50,1,0,end,end
7,[Amend Regulations 8 and 10 of the principal R...,277,743,2034,788,3,1757,1,2,middle,middle
8,time during which maternity allowance may be p...,275,791,1295,835,3,1020,1,0,end,end
9,5/VI/1986 Social Security (Benefits) (Age Pens...,208,860,2027,904,3,1819,0,2,start,start


In [8]:
l.loc[lli_df["page"]==4]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
55,1987—List No. 3 4 (110),431,222,2245,272,4,1814,0,2,start
56,ARGENTINA (cont.),416,367,831,404,4,415,0,0,country
57,"20,XI,1985 Decree No. 2224 to set up a provisi...",416,431,2246,473,4,1830,0,2,start
58,applications for the revision of collective ba...,481,477,2242,523,4,1761,1,2,middle
59,Ministry of Labour and Social Security. (ibid....,484,527,1556,568,4,1072,1,0,end
60,(The Decree refers to the austerity programme ...,485,595,2242,638,4,1757,1,2,middle
61,reduce inflation. It indicates that existing c...,484,641,2247,685,4,1763,1,2,middle
62,date because of developments in the past years...,483,688,2242,731,4,1759,1,2,middle
63,"acceptances. under the Ministry of Labour, wit...",483,736,2242,779,4,1759,1,2,middle
64,interests with the need for the revision of su...,484,783,2242,825,4,1758,1,2,middle


In [9]:
lli_df.loc[lli_df["page"]==5]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
107,(1th) 5 1987—List No. 3,220,242,2039,293,5,1819,0,2,start,start
108,AUSTRALIA,217,385,623,425,5,406,0,0,country,country
109,11/11/1986 Notice regarding approval of accomm...,217,452,2029,495,5,1812,0,2,start,start
110,No. § 111.,277,501,477,533,5,200,1,0,end,end
111,{Issued under the Aged or Disabled Persons Hom...,281,569,1482,611,5,1201,1,0,end,end
112,1/1V/1986 Public Service Board Notice No. S 13...,220,639,2033,681,5,1813,0,2,start,start
113,to officers appointed and employees engaged un...,281,685,1807,729,5,1526,1,1,end,end
114,[Sets forth general provisions and procedures ...,284,753,2034,796,5,1750,1,2,middle,middle
115,and loss of essential qualifications. Issued p...,281,800,2034,843,5,1753,1,2,middle,middle
116,(Redeployment and Retirement) Act 1979.],280,845,1107,888,5,827,1,0,end,end


In [10]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec.head(20)

,country,region,text,page
0,ANTIGUA AND BARBUDA,,5/V1/1986 Social Security (Benefits) (Invalidi...,3
1,ANTIGUA AND BARBUDA,,5/V1/1986 Social Security (Benefits) (Maternit...,3
2,ANTIGUA AND BARBUDA,,5/VI/1986 Social Security (Benefits) (Age Pens...,3
3,ARGENTINA,,9/VIII/1985 Decision No. 1280 concerning the a...,3
4,ARGENTINA,,13/IX/1985 Act No. 23226 to amend s. 38 of Act...,3
5,ARGENTINA,,30/1X/1985 Act No. 23302 on policy-making with...,3
6,ARGENTINA,,2/X/1985 Act No. 23238 respecting the rehabili...,3
7,ARGENTINA,,1/X1/1985 Decree No. 2131 to establish the Com...,3
8,ARGENTINA,,1987—List No. 3 4 (110),4
9,ARGENTINA,,"20,XI,1985 Decree No. 2224 to set up a provisi...",4


In [17]:
rec.loc[rec["page"]==8]

,country,region,text,page
51,AUSTRIA,,1987—List No. 3 8 (114),8
52,AUSTRIA,,20/II1/1969 Federal Act amending the Act of 25...,8
53,AUSTRIA,,25/V1/1986 New amendment to the Act regarding ...,8
54,AUSTRIA,,3/VII/1986 Federal Act of 3 July 1986 amending...,8
55,AUSTRIA,,3/VII/1986 Federal Act amending the Workers’ P...,8
56,BANGLADESH,,13/11/1985 Industrial Relations (Amendment) Or...,8
57,BANGLADESH,,13/11/1985 The Employment of Labour (Standing ...,8
58,BARBADOS,,25/V11/1985 National Insurance and Social Secu...,8


In [70]:
rec.to_csv("index_1944.csv")

In [12]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 10)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df

## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944 seiten 8-9, schlechte Ländererkennung
- 1921 Seite 7
- 1986 Seiten 4-7 FIXED

## Fragen
